# Data wrangling and Analysis about OpenStreetMap in Zurich:
1. Data wrangling process
2. Import data
3. Overview of data files
4. Overview of users
5. Data creation
 - Timeline 
 - The number of ways and nodes changed by different computer programs
6. Overview of nodes and ways
 - What are the most common keys for node_tags and way_tags?
 - What are the most common values for node_tags and way_tags?
 - Who contribute most for nodes and ways?
 - How many tags for one node or one tag? 
 - What are the nodes with most and second most tags?
 - What are the ways with most and second most tags?
7. Explore the map of Zurich
 - Type of street names
 - Number of data in each canton and each district
 - Restaurants types, locations and opening on Sunday
 - Most seen natural scene
 - The most common area code for phone number
 - The most common postcode in the address
8. Appendix
 - Decisions made in wrangling data 
 - Fields without consistent format and didn't process
 - Suggestions for improving and analyzing the data

## Data Wrangling process:
1. Use `get_element.py` to generate data with only 1/10 and 1/100000 size of original data.
2. Use `key_overview.py` to generate to look what are the keys and the values as a reference for further processing.
3. Use `exam.py` to look what types of value would appear in the tag I am going to modify. 
4. Functions used for data processing are all in `clean_format.py` and `is_in.py`.
5. Use `functions.py` to run the whole process.
6. Out put files are:
 - nodes.csv
 - nodes_tags.csv
 - ways.csv
 - ways_nodes.csv
 - ways_tags.csv
7. use sqlite3 to combine all csv files into a data base: `OpenStreetMap.db`
8. Do exploratory data analysis using Jupyter notebook.  

## Import data

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline
import seaborn as sns
import plotly.plotly as py # interactive graphing
import plotly.tools as tls
tls.set_credentials_file(username='XXXXX', api_key='XXXXXX')
tls.embed("https://plot.ly/~streaming-demos/4")
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
from plotly.graph_objs import Bar, Scatter, Marker, Layout 
import plotly.graph_objs as go
from time import time
import re

import requests
from bs4 import BeautifulSoup
from collections import defaultdict
import gmaps
import gmaps.datasets
api='XXXX'

db = sqlite3.connect("HH2.db")
# another way to initialize database:
# db = create_engine('sqlite:///311_8M.db') # Initializes database with filename 311_8M.db in current directory
# use cursor
# c = db.cursor()
# c.execute("select * from ways_tags where key='highway'")
# c.fetchall()

## Overview of data files:
* nodes.csv: 212.5 MB
* nodes_tags.csv: 15.9 MB
* ways.csv: 24.6 MB
* ways_tags.csv: 43.1 MB
* ways_nodes.csv: 77.6 MB
* OpenStreetMap.db (database includes all files above): 331.5 MB
* raw osm file: 611.5 MB

In [ ]:
## number of lines in each file
line_nodes = pd.read_sql_query('SELECT count() FROM nodes',db)['count()']
line_nodes_tags = pd.read_sql_query('SELECT count() FROM nodes_tags',db)['count()']
line_ways = pd.read_sql_query('SELECT count() FROM ways',db)['count()']
line_ways_tags = pd.read_sql_query('SELECT count() FROM ways_tags',db)['count()']
line_ways_nodes = pd.read_sql_query('SELECT count() FROM ways_nodes',db)['count()']
dic_data = {'nodes':line_nodes,'nodes_tags':line_nodes_tags,'ways':line_ways,'ways_tags':line_ways_tags,'ways_nodes':line_ways_nodes}
df_data = pd.DataFrame(dic_data)
df_data

## Overview of users:

In [ ]:
c = db.cursor()
# number of unique users:
c.execute('SELECT count(uid) FROM (SELECT uid FROM nodes UNION SELECT uid FROM ways)')
print 'Number of unique users:', c.fetchall()
# number of users contribute only once:
c.execute('SELECT count() FROM\
          (SELECT uid, count() as num \
          FROM (SELECT uid, id FROM nodes UNION ALL SELECT uid, id FROM ways)\
          GROUP BY uid ORDER BY num DESC)  \
          WHERE num=1' 
         )
print 'Number of users only contribute once', c.fetchall()
# users contribute most to nodes:
user_node = pd.read_sql_query('SELECT user, count() as num FROM nodes GROUP BY uid ORDER BY num DESC LIMIT 10',db)
# users contribute most to ways:
user_way = pd.read_sql_query('SELECT user, count() as num FROM ways GROUP BY uid ORDER BY num DESC LIMIT 10',db)
# plot
graph1 = go.Bar(x=user_node.user, y=user_node.num, name='node-user')
graph2 = go.Bar(x=user_way.user, y=user_way.num, name='way-user')
fig = tls.make_subplots(rows=1, cols=2, subplot_titles=('node', 'way'))
fig.append_trace(graph1, 1, 1)
fig.append_trace(graph2, 1, 2)
fig['layout'].update(height=400, width=800, title='Users contribute most for nodes and ways')
py.iplot(fig, filename='most-common-user')

## Data creation
### Timeline 

In [ ]:
## timestamp of creating nodes
nodes_time = pd.read_sql_query('SELECT timestamp FROM nodes',db)
nodes_time['time'] = nodes_time['timestamp'].str.replace('[\-\d]{3}T.*Z','')
create_time_nodes = nodes_time.groupby('time').count().reset_index()
## timestamp of creating ways 
ways_time = pd.read_sql_query('SELECT timestamp FROM ways',db)
ways_time['time'] = ways_time['timestamp'].str.replace('[\-\d]{3}T.*Z','')
create_time_ways = ways_time.groupby('time').count().reset_index()
create_time = pd.concat([create_time_ways,create_time_nodes])
## plot
graph1 = go.Bar(x=create_time.time, y=create_time.timestamp, name='node')
graph2 = go.Bar(x=create_time.time, y=create_time.timestamp, name='way')
fig = tls.make_subplots(rows=1, cols=2)
fig.append_trace(graph1, 1, 1)
fig.append_trace(graph2, 1, 2)
fig['layout'].update(height=400, width=1000, title='Time distribution of creating nodes and ways')
py.iplot(fig, filename='created-time-nodes-ways')

### The number of ways and nodes changed by different computer programs 

In [ ]:
## number of nodes
c.execute('SELECT count() FROM nodes')
print 'Total number of nodes:', c.fetchall()
## number of ways
c.execute('SELECT count() FROM ways')
print 'Total number of ways:', c.fetchall()

### What are the most common keys for node_tags and way_tags?

In [ ]:
# most common key for nodes
most_tags_node = pd.read_sql_query('SELECT key, count() as num FROM nodes_tags GROUP BY key ORDER BY num DESC LIMIT 10',db)
# most common key for ways
most_tags_way = pd.read_sql_query('SELECT key, count() as num FROM ways_tags GROUP BY key ORDER BY num DESC LIMIT 10',db)
# plot
# py.iplot([Bar(x=most_tags_node.key, y=most_tags_node.num)], filename='most common tags_key for node')
graph1 = go.Bar(x=most_tags_node.key, y=most_tags_node.num, name='node-key')
graph2 = go.Bar(x=most_tags_way.key, y=most_tags_way.num, name='way-key')
fig = tls.make_subplots(rows=1, cols=2, subplot_titles=('node', 'way'))
fig.append_trace(graph1, 1, 1)
fig.append_trace(graph2, 1, 2)
fig['layout'].update(height=400, width=800, title='Most common key for node tags and way tags')
py.iplot(fig, filename='most-common-tags-key')

### What are the most common values for node_tags and way_tags?

In [ ]:
most_value_node = pd.read_sql_query('SELECT key, value, count() as num FROM nodes_tags GROUP BY value ORDER BY num DESC LIMIT 10',db)
most_value_node['key-value'] = most_value_node.key.str.cat(most_value_node.value, sep='-')
most_value_way = pd.read_sql_query('SELECT key, value, count() as num FROM ways_tags GROUP BY value ORDER BY num DESC LIMIT 10',db)
most_value_way['key-value'] = most_value_way.key.str.cat(most_value_way.value, sep='-')

graph1 = go.Bar(x=most_value_node['key-value'], y=most_tags_node.num, name='node-value')
graph2 = go.Bar(x=most_value_way['key-value'], y=most_tags_way.num, name='way-value')
fig = tls.make_subplots(rows=1, cols=2, subplot_titles=('node', 'way'))
fig.append_trace(graph1, 1, 1)
fig.append_trace(graph2, 1, 2)
fig['layout'].update(height=400, width=800, title='Most common value for node tags and way tags (key-value)')
py.iplot(fig, filename='most-common-tags-value')



### How many tags for one node or one tag?

In [ ]:
node_num = pd.read_sql_query('SELECT nodes.id, count() as num \
                             FROM nodes LEFT JOIN nodes_tags \
                             ON nodes.id=nodes_tags.id \
                             GROUP BY nodes.id ORDER BY num DESC',db)
way_num = pd.read_sql_query('SELECT ways.id, count() as num \
                             FROM ways LEFT JOIN ways_tags \
                             ON ways.id=ways_tags.id \
                             GROUP BY ways.id ORDER BY num DESC',db)
print 'Top 5 nodes with most tags:'
print node_num.head()
print 
print 'Top 5 ways with most tags:'
print way_num.head()

fig, (ax1,ax2,ax3) = plt.subplots(1,3,figsize=(12,3))
g=sns.countplot(x='num',data=node_num,ax=ax1)
g.set(xlim=(-0.5,16),xlabel='Number of Nodes')
g.set(title='Number of Tags in One Node \n max node number ='+str(node_num['num'][0]),ylabel='Count (%)')
g=sns.countplot(x='num',data=node_num,ax=ax2)
g.set(xlim=(-0.5,16))
g.set(ylim=(0,50000))
g.set(title='Zoom in \n- Number of Tags in One Node',xlabel='Number of Nodes')
g=sns.countplot(x='num',data=way_num,ax=ax3)
g.set(xlim=(-0.5,16))
g.set(title='Number of Tags in One Way \n max way number ='+str(way_num['num'][0]),xlabel='Number of Ways')
# node_num = pd.read_sql_query('SELECT nodes.id, count() as num FROM nodes LEFT JOIN nodes_tags ON nodes.id=nodes_tags.id GROUP BY nodes.id ORDER BY num DESC',db)

### What's node with most tags

In [ ]:
 pd.read_sql_query('SELECT type, key, value \
                   FROM nodes_tags\
                   WHERE id IN \
                   (SELECT nID FROM\
                   (SELECT nodes.id as nID, count() as num\
                   FROM nodes LEFT JOIN nodes_tags \
                   ON nodes_tags.id= nodes.id\
                   GROUP BY nodes.id\
                   ORDER BY num DESC\
                   LIMIT 1))',db)

### What is the node with the second most tags

In [ ]:
# see the tags for the second largest node
pd.read_sql_query('SELECT type, key, value\
                   FROM nodes_tags\
                   WHERE id in \
                   (SELECT nID FROM\
                   (SELECT nodes.id as nID, count() as num\
                   FROM nodes LEFT JOIN nodes_tags \
                   ON nodes_tags.id= nodes.id\
                   GROUP BY nodes.id\
                   ORDER BY num DESC\
                   LIMIT 1 OFFSET 1))',db)

### What is the way with most tags?

In [ ]:
pd.read_sql_query('SELECT type, key, value \
                   FROM ways_tags\
                   WHERE id IN \
                   (SELECT wID FROM\
                   (SELECT ways.id as wID, count() as num\
                   FROM ways LEFT JOIN ways_tags \
                   ON ways_tags.id= ways.id\
                   GROUP BY ways.id\
                   ORDER BY num DESC\
                   LIMIT 1))',db)

### What is the way with second most tags?

In [ ]:
pd.read_sql_query('SELECT type, key, value \
                   FROM ways_tags\
                   WHERE id IN \
                   (SELECT wID FROM\
                   (SELECT ways.id as wID, count() as num\
                   FROM ways LEFT JOIN ways_tags \
                   ON ways_tags.id= ways.id\
                   GROUP BY ways.id\
                   ORDER BY num DESC\
                   LIMIT 1 OFFSET 1))',db)

In [ ]:
pd.read_sql_query('SELECT type, key, value \
                   FROM ways_tags\
                   WHERE id IN \
                   (SELECT wID FROM\
                   (SELECT ways.id as wID, count() as num\
                   FROM ways LEFT JOIN ways_tags \
                   ON ways_tags.id= ways.id\
                   GROUP BY ways.id\
                   ORDER BY num DESC\
                   LIMIT 1 OFFSET 4))',db)

** In general, more tags were used to describe nodes than ways. The first two nodes with most tags are the city of Zurich and Zurich airport. Most of the tags are used to describe the name in different languages. For tags of way, the most tagged way is a recycling center. Most tags are used to describe the material for recycling. The 2nd most tagged way is a hotel, with tags descrbing the way of payment. The 3rd most tagged way is a gas station, with tags descrbing the provided fuel. ** 

## Explore Zurich
### The type of street names in Zurich

In [ ]:
count_strasse = pd.read_sql_query('SELECT SUM(num) FROM\
                                  (SELECT count(DISTINCT value) as num FROM nodes_tags WHERE UPPER(value) LIKE "%STRASSE%"\
                                  UNION ALL\
                                  SELECT count(DISTINCT value) as num FROM ways_tags WHERE UPPER(value) LIKE "%STRASSE%")\
                                  ',db)
count_weg = pd.read_sql_query('SELECT SUM(num) FROM\
                                  (SELECT count(DISTINCT value) as num FROM nodes_tags WHERE UPPER(value) LIKE "%WEG%"\
                                  UNION ALL\
                                  SELECT count(DISTINCT value) as num FROM ways_tags WHERE UPPER(value) LIKE "%WEG%")\
                                  ',db)
count_platz = pd.read_sql_query('SELECT SUM(num) FROM\
                                  (SELECT count(DISTINCT value) as num FROM nodes_tags WHERE UPPER(value) LIKE "%PLATZ%"\
                                  UNION ALL\
                                  SELECT count(DISTINCT value) as num FROM ways_tags WHERE UPPER(value) LIKE "%PLATZ%")\
                                  ',db)
count_gasse = pd.read_sql_query('SELECT SUM(num) FROM\
                                  (SELECT count(DISTINCT value) as num FROM nodes_tags WHERE UPPER(value) LIKE "%GASSE%"\
                                  UNION ALL\
                                  SELECT count(DISTINCT value) as num FROM ways_tags WHERE UPPER(value) LIKE "%GASSE%")\
                                  ',db)
street_type = ['strasse','weg','platz','gasse']
street_count = [count_strasse['SUM(num)'][0],count_weg['SUM(num)'][0],count_platz['SUM(num)'][0],count_gasse['SUM(num)'][0]]

ind = np.arange(len(street_count))  # the x locations for the groups
width = 0.35       # the width of the bars
plt.figure(figsize=(4,3))
plt.bar(range(len(street_count)), street_count, width, color='b')

plt.ylabel('Counts')
plt.title('Type of Street Names in Zurich')
plt.xticks(ind + width / 2, street_type,  rotation='vertical')


### Number of data in each canton and each district

In [ ]:
canton = pd.read_sql_query('SELECT value, count(DISTINCT id) FROM\
                        (SELECT id, value FROM nodes_tags WHERE key="canton"\
                        UNION ALL\
                        SELECT id, value FROM ways_tags WHERE key="canton")\
                        GROUP BY value',db)
print 'number of data in each canton:'
print canton
print
district = pd.read_sql_query('SELECT value, count(DISTINCT id) FROM\
                        (SELECT id, value FROM nodes_tags WHERE key="district"\
                        UNION ALL\
                        SELECT id, value FROM ways_tags WHERE key="district")\
                        GROUP BY value',db)
print 'number of data in each district:'
print district

** Most of the data came from Zurich (52%). 34% data came from another canton: Aargau. 14% data contain too few information to find its canton.**

### Restaurants types, locations and opening on Sunday

In [ ]:
create = pd.read_sql_query('SELECT value, count()as num FROM\
                            (SELECT id, value FROM nodes_tags WHERE key="created_by" \
                            UNION ALL\
                            SELECT id, value FROM ways_tags WHERE key="created_by")\
                            GROUP BY value ORDER BY num DESC',db)
create

## Overview of nodes and ways
### Total number of ways and nodes

In [ ]:
## Number of resturants with more than 5 shops
cuisine = pd.read_sql_query('SELECT value, count() as num FROM \
                            (SELECT value, id FROM nodes_tags WHERE key="cuisine" \
                            UNION ALL SELECT value, id FROM ways_tags WHERE key="cuisine") a \
                             GROUP BY value HAVING num>5 ORDER BY num DESC',db)
plt.figure(figsize=(10,3))
g = sns.barplot(x='value',y='num',data=cuisine)
g.set_xticklabels(cuisine.value, rotation=90)
g.set(title='Restaurant Types (at least 5 restaurants)',xlabel='',ylabel='Count')

In [ ]:
## group these types of restaurants, to see the number:
# 1. regional, swiss, local
# 2. italian, pizza
# 3. asian, chinese, thai, vietnamese
# 4. japanese, sushi
# 5. american, burger
# 6. indian
# 7. turkish, kebab, falafel
swiss = pd.read_sql_query('SELECT value, count() FROM nodes_tags WHERE key="cuisine" AND\
                          (UPPER(value) LIKE "%REGIONAL%" OR UPPER(value) LIKE "%SWISS%" OR UPPER(value) LIKE "%LOCAL%")',db)
pizza = pd.read_sql_query('SELECT value,count() FROM nodes_tags WHERE key="cuisine" AND\
                          (UPPER(value) LIKE "%ITALIAN%" OR UPPER(value) LIKE "%PIZZA%")',db)
asian = pd.read_sql_query('SELECT value,count() FROM nodes_tags WHERE key="cuisine" AND\
                          (UPPER(value) LIKE "%VIETNA%" OR UPPER(value) LIKE "%CHINESE%" OR UPPER(value) LIKE "%THAI%" OR UPPER(value) LIKE "%ASIAN%")',db)
japan = pd.read_sql_query('SELECT value,count() FROM nodes_tags WHERE key="cuisine" AND\
                          (UPPER(value) LIKE "%JAPAN%" OR UPPER(value) LIKE "%SUSHI%")',db)
usa = pd.read_sql_query('SELECT value,count() FROM nodes_tags WHERE key="cuisine" AND\
                          (UPPER(value) LIKE "%AMERICA%" OR UPPER(value) LIKE "%BURGER%")',db)
indian = pd.read_sql_query('SELECT value,count() FROM nodes_tags WHERE key="cuisine" AND\
                          (UPPER(value) LIKE "%INDIAN%")',db)
kebab = pd.read_sql_query('SELECT value,count() FROM nodes_tags WHERE key="cuisine" AND\
                          (UPPER(value) LIKE "%TURKISH%" OR UPPER(value) LIKE "%KEBAB%" OR UPPER(value) LIKE "%FALAFEL%")',db)

type_cuisine = pd.concat([swiss,pizza,asian,japan,usa,indian,kebab])
type_cuisine.loc[type_cuisine['value']=='vietnamese','value']='Asian'
plt.figure(figsize=(8,3))
g = sns.barplot(x='value',y='count()',data=type_cuisine)
g.set(xlabel='',title='Number of restaurants grouped by cuisine types')


In [ ]:
## Types and number of restaurant opens on Sunday
sun_open_restaurant = pd.read_sql_query('SELECT value, count() as num FROM\
                   (SELECT id, value FROM nodes_tags WHERE id IN\
                   (SELECT id FROM \
                   (SELECT id, key, value FROM nodes_tags WHERE id IN (SELECT id FROM nodes_tags WHERE key="cuisine")) a\
                   WHERE key="opening_hours" AND value LIKE "%Su ____%") \
                   AND key="cuisine" \
                   UNION ALL\
                   SELECT id, value FROM ways_tags WHERE id IN\
                   (SELECT id FROM \
                   (SELECT id, key, value FROM ways_tags WHERE id IN (SELECT id FROM ways_tags WHERE key="cuisine")) a\
                   WHERE key="opening_hours" AND value LIKE "%Su ____%") \
                   AND key="cuisine")\
                   GROUP BY value ORDER BY num DESC',db)
plt.figure(figsize=(10,3))
g = sns.barplot(x='value',y='num',data=sun_open_restaurant)
g.set_xticklabels(sun_open_restaurant.value,rotation=90)
g.set(title='Restaurants open on Sunday',xlabel='',ylabel='Count')

In [ ]:
lat_lon_italy = pd.read_sql_query('SELECT lat, lon FROM nodes WHERE id IN\
                  (SELECT id FROM nodes_tags WHERE key="cuisine" AND (UPPER(value) LIKE "%ITALIAN%" OR UPPER(value) LIKE "%PIZZA%"))\
                  ',db)
lat_lon_regional = pd.read_sql_query('SELECT lat, lon FROM nodes WHERE id IN\
                  (SELECT id FROM nodes_tags WHERE key="cuisine" AND (UPPER(value) LIKE "%REGIONAL%" OR UPPER(value) LIKE "%LOCAL%"))\
                  ',db)
lat_lon_sunday = pd.read_sql_query("SELECT lat, lon FROM nodes WHERE id IN\
                                   (SELECT id FROM \
                                   (SELECT id, key, value FROM nodes_tags WHERE id IN\
                                   (SELECT id FROM nodes_tags WHERE key='cuisine'))\
                                   WHERE key='opening_hours' AND value LIKE '%Su ____%')",db)

## plot gmap
italy = [tuple(x) for x in lat_lon_italy.values]
regional = [tuple(x) for x in lat_lon_regional.values]
sunday = [tuple(x) for x in lat_lon_sunday.values]
gmaps.configure(api_key=api) # Your Google API key

italy_layer = gmaps.symbol_layer(italy, fill_color="blue", stroke_color="blue")
regional_layer = gmaps.symbol_layer(regional, fill_color="red", stroke_color="red")
sunday_layer = gmaps.symbol_layer(sunday, fill_color="green", stroke_color="green")
m = gmaps.Map()
m.add_layer(italy_layer)
m.add_layer(regional_layer)
m.add_layer(sunday_layer)
m


** 1. If we regards pizza as italian food, then Italian food would be the most common cuisine in Zurich. regional cuisine would be the second, then asian cuisine. Italian (blue points) and regional cuisine (red points) doesn't localize at certain region. **

** 2. Only less than 10% of restaurnats open on Sunday (green points). These restaurants does not localize at certain region either. **

## Most seen natural scene

In [ ]:
## Most seen natural scene
natural_scene = pd.read_sql_query('SELECT value, count() as num FROM\
                   (SELECT value, id FROM nodes_tags WHERE key="natural" \
                   UNION ALL\
                   SELECT value, id FROM ways_tags WHERE key="natural")\
                   GROUP BY value ORDER BY num DESC',db)
print '#1 of most seen natural scene:',natural_scene.loc[0,'value'] ,'with number:',natural_scene.loc[0,'num']
plt.figure(figsize=(12,3))
g = sns.barplot(x='value',y='num',data=natural_scene[1:])
g.set_xticklabels(natural_scene.value,rotation=90)
g.set(title='Most seen natural scene',xlabel='',ylabel='Count')

### The most common  postcode in the address